# Supplement Experiments

1001-Experiments makes personalized supplements tailored to individual health needs.

1001-Experiments aims to enhance personal health by using data from wearable devices and health apps.

This data, combined with user feedback and habits, is used to analyze and refine the effectiveness of the supplements provided to the user through multiple small experiments.

The data engineering team at 1001-Experiments plays a crucial role in ensuring the collected health and activity data from thousands of users is accurately organized and integrated with the data from supplement usage. 

This integration helps 1001-Experiments provide more targeted health and wellness recommendations and improve supplement formulations.


## Task

1001-Experiments currently has the following four datasets with four months of data:
 - "user_health_data.csv" which logs daily health metrics, habits and data from wearable devices,
 - "supplement_usage.csv" which records details on supplement intake per user,
 - "experiments.csv" which contains metadata on experiments, and
 - "user_profiles.csv" which contains demographic and contact information of the users.

Each dataset contains unique identifiers for users and/or their supplement regimen.

The developers and data scientsits currently manage code that cross-references all of these data sources separately, which is cumbersome and error-prone.

Your manager has asked you to write a Python function that cleans and merges these datasets into a single dataset.

The final dataset should provide a comprehensive view of each user's health metrics, supplement usage, and demographic information.

- To test your code, your manager will run only the code `merge_all_data('user_health_data.csv', 'supplement_usage.csv', 'experiments.csv', 'user_profiles.csv')`
- Your `merge_all_data` function must return a DataFrame, with columns as described below.
- All columns must accurately match the descriptions provided below, including names.


## Data

The provided data is structured as follows:

![database schema](schema.png)

The function you write should return data as described below.

There should be a unique row for each daily entry combining health metrics and supplement usage.

Where missing values are permitted, they should be in the default Python format unless stated otherwise.

| Column Name        | Description |
|--------------------|-------------|
| user_id            | Unique identifier for each user. </br>There should not be any missing values. |
| date               | The date the health data was recorded or the supplement was taken, in date format. </br>There should not be any missing values. |
| email              | Contact email of the user. </br>There should not be any missing values. |
| user_age_group  | The age group of the user, one of: 'Under 18', '18-25', '26-35', '36-45', '46-55', '56-65', 'Over 65' or 'Unknown' where the age is missing.|
| experiment_name    | Name of the experiment associated with the supplement usage. </br>Missing values for users that have user health data only is permitted. |
| supplement_name    | The name of the supplement taken on that day. Multiple entries are permitted. </br>Days without supplement intake should be encoded as 'No intake'. |
| dosage_grams       | The dosage of the supplement taken in grams. Where the dosage is recorded in mg it should be converted by division by 1000.</br>Missing values for days without supplement intake are permitted. |
| is_placebo         | Indicator if the supplement was a placebo (true/false). </br>Missing values for days without supplement intake are permitted. |
| average_heart_rate | Average heart rate as recorded by the wearable device. </br>Missing values are permitted. |
| average_glucose    | Average glucose levels as recorded on the wearable device. </br>Missing values are permitted. |
| sleep_hours        | Total sleep in hours for the night preceding the current day’s log. </br>Missing values are permitted. |
| activity_level     | Activity level score between 0-100. </br>Missing values are permitted. |

In [120]:
# Use as many python cells as you wish to write your code

In [121]:
import pandas as pd
import numpy as np

In [122]:
# Merging the data in a single database

def merge_all_data(user_health_data_path, supplement_usage_path, experiments_path, user_profiles_path):

    user_profiles = pd.read_csv(user_profiles_path)
    user_health_data = pd.read_csv(user_health_data_path)
    supplement_usage = pd.read_csv(supplement_usage_path)
    experiments = pd.read_csv(experiments_path)

    #Adding age categories to user_profiles and remove the 'age' column

    user_profiles["user_age_group"] = np.where(user_profiles['age'] < 18, 'Under 18',
                                    np.where(user_profiles['age'] <= 25, '18-25',
                                    np.where(user_profiles['age'] <= 35, '26-35',
                                    np.where(user_profiles['age'] <= 45, '36-45', 
                                    np.where(user_profiles['age'] <= 55, '46-55',
                                    np.where(user_profiles['age'] <= 65, '56-65',
                                    np.where(user_profiles['age'] > 65, 'Over 65', 'Unknown')))))))

    user_profiles.drop(columns=['age'], inplace=True)

    # Converting date to datetime

    user_health_data["date"] = pd.to_datetime(user_health_data['date']).dt.date
    supplement_usage["date"] = pd.to_datetime(supplement_usage['date']).dt.date

    # Renaming the column to experiment name
    experiments.rename(columns={'name': 'experiment_name'}, inplace=True)
    experiments.drop(columns=['description'], inplace=True)
    
    supplement_usage["dosage_grams"] = supplement_usage["dosage"]/1000
    
    supplement_usage.drop(columns=['dosage', 'dosage_unit'], inplace=True)

    # Cleaning the user health data columns

    user_health_data["sleep_hours"] = user_health_data["sleep_hours"].str.replace('h','', case=False).astype(float)

    activity_map = {
        1: 0,
        2: 33,
        3: 67,
        4: 100
        }
    
    user_health_data['activity_level'] = user_health_data['activity_level'].map(activity_map)

    user_profile_health = user_profiles.merge(user_health_data, on="user_id")
    supplement_exp = supplement_usage.merge(experiments, on="experiment_id")

    dataframe = user_profile_health.merge(supplement_exp, on=["user_id", "date"], how='left')
    
    dataframe.drop(columns=['experiment_id'], inplace=True)
    dataframe["supplement_name"] = dataframe["supplement_name"].fillna('No intake')
    
    return dataframe